In [3]:
import numpy as np
from numpy.linalg import det, inv
import os

# ---------- Gaussian PDF ----------
def gaussian_pdf(x, mean, cov):
    d = len(x)
    diff = x - mean
    num = np.exp(-0.5 * diff.T @ inv(cov) @ diff)
    den = np.sqrt((2 * np.pi) ** d * det(cov))
    return num / (den + 1e-12)

# ---------- Simple K-Means ----------
def kmeans(X, K, tol=1e-4):
    N, D = X.shape
    np.random.seed(0)
    centroids = X[np.random.choice(N, K, replace=False)]
    labels = np.zeros(N, dtype=int)
    while True:
        distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
        new_labels = np.argmin(distances, axis=1)
        new_centroids = np.array([X[new_labels == k].mean(axis=0) for k in range(K)])
        if np.linalg.norm(new_centroids - centroids) < tol:
            break
        centroids, labels = new_centroids, new_labels
    return centroids, new_labels

# ---------- GMM EM ----------
def gmm_em(X, K, tol=1e-4):
    N, D = X.shape
    means, labels = kmeans(X, K)
    covs = np.zeros((K, D, D))
    weights = np.zeros(K)
    for k in range(K):
        Xk = X[labels == k]
        if len(Xk) == 0:
            covs[k] = np.eye(D)
        else:
            covs[k] = np.cov(Xk, rowvar=False) + 1e-6 * np.eye(D)
        weights[k] = len(Xk) / N

    prev_log_likelihood = -np.inf
    while True:
        # E-step
        resp = np.zeros((N, K))
        for k in range(K):
            resp[:, k] = weights[k] * np.array([gaussian_pdf(x, means[k], covs[k]) for x in X])
        resp_sum = resp.sum(axis=1, keepdims=True)
        resp /= resp_sum

        # M-step
        Nk = resp.sum(axis=0)
        for k in range(K):
            means[k] = (resp[:, k][:, None] * X).sum(axis=0) / Nk[k]
            diff = X - means[k]
            covs[k] = (resp[:, k][:, None, None] * np.einsum('ni,nj->nij', diff, diff)).sum(axis=0) / Nk[k]
            covs[k] += 1e-6 * np.eye(D)
            weights[k] = Nk[k] / N

        # Convergence
        log_likelihood = np.sum(np.log(resp_sum))
        if abs(log_likelihood - prev_log_likelihood) < tol:
            break
        prev_log_likelihood = log_likelihood

    return means, covs, weights


# ---------- Naive Bayes + GMM ----------
class NaiveBayesGMM:
    def __init__(self, n_components=2):
        self.n_components = n_components
        self.models = {}
        self.class_priors = {}

    def fit(self, X, y):
        classes = np.unique(y)
        for cls in classes:
            Xc = X[y == cls]
            means, covs, weights = gmm_em(Xc, self.n_components)
            self.models[cls] = (means, covs, weights)
            self.class_priors[cls] = len(Xc) / len(X)

    def predict(self, X):
        preds = []
        for x in X:
            class_probs = {}
            for cls, (means, covs, weights) in self.models.items():
                p_x_given_cls = np.sum([
                    weights[k] * gaussian_pdf(x, means[k], covs[k])
                    for k in range(self.n_components)
                ])
                p_cls = self.class_priors[cls]
                class_probs[cls] = p_x_given_cls * p_cls
            preds.append(max(class_probs, key=class_probs.get))
        return np.array(preds)

    # ---------- Save Model ----------
    def save(self, folder_path="gmm_model"):
        os.makedirs(folder_path, exist_ok=True)
        np.save(os.path.join(folder_path, "class_priors.npy"), self.class_priors)
        np.save(os.path.join(folder_path, "n_components.npy"), self.n_components)
        # Save each class model separately
        for cls, (means, covs, weights) in self.models.items():
            np.save(os.path.join(folder_path, f"class_{cls}_means.npy"), means)
            np.save(os.path.join(folder_path, f"class_{cls}_covs.npy"), covs)
            np.save(os.path.join(folder_path, f"class_{cls}_weights.npy"), weights)

    # ---------- Load Model ----------
    def load(self, folder_path="gmm_model"):
        self.n_components = int(np.load(os.path.join(folder_path, "n_components.npy")))
        self.class_priors = np.load(os.path.join(folder_path, "class_priors.npy"), allow_pickle=True).item()
        self.models = {}
        for cls in self.class_priors.keys():
            means = np.load(os.path.join(folder_path, f"class_{cls}_means.npy"))
            covs = np.load(os.path.join(folder_path, f"class_{cls}_covs.npy"))
            weights = np.load(os.path.join(folder_path, f"class_{cls}_weights.npy"))
            self.models[cls] = (means, covs, weights)

def load_train_test_datasets(base_folder):
    def load_folder(folder):
        data = []
        labels = []
        class_map = {}  # Maps class name to label (e.g. 'class1' → 0)

        files = [f for f in os.listdir(folder) if f.endswith(".txt")]
        files.sort()  # Optional: for consistent ordering

        for file in files:
            path = os.path.join(folder, file)
            parts = file.split("_")
            class_name = parts[0]  # 'class1' from 'class1_train.txt'

            # Assign numeric label to class name
            if class_name not in class_map:
                class_map[class_name] = len(class_map)  # auto-labeling

            label = class_map[class_name]

            points = np.loadtxt(path, delimiter=' ')
            data.append(points)
            labels.append(np.full(len(points), label))

        return np.vstack(data), np.hstack(labels), class_map

    train_folder = os.path.join(base_folder, "train")
    test_folder = os.path.join(base_folder, "test")

    X_train, y_train, class_map = load_folder(train_folder)
    X_test, y_test, _ = load_folder(test_folder)  # test uses same class_map, but we ignore it

    return X_train, y_train, X_test, y_test, class_map

In [4]:
X_train, y_train, X_test, y_test, class_map = load_train_test_datasets(r'../../Dataset/Group04/NLS_Group04/')

nb_gmm = NaiveBayesGMM(n_components=2)
nb_gmm.fit(X_train, y_train)
nb_gmm.save("Dataset")

print("✅ Model saved successfully.")

✅ Model saved successfully.


In [5]:
loaded_model = NaiveBayesGMM()
loaded_model.load("my_gmm_model")

y_pred = loaded_model.predict(X_train)
acc = np.mean(y_pred == y_train)
print(f"Reloaded model accuracy: {acc:.3f}")


Reloaded model accuracy: 1.000


In [6]:
import numpy as np
from numpy.linalg import det, inv, slogdet
import os
import matplotlib.pyplot as plt

# -----------------------------
# 1) Gaussian PDF (multivariate)
# -----------------------------
def gaussian_pdf(x, mean, cov):
    """Return multivariate Gaussian density value at x."""
    d = mean.shape[0]
    diff = x - mean
    # use slogdet for numeric stability
    sign, logdet = slogdet(cov)
    if sign <= 0:
        cov = cov + 1e-6 * np.eye(d)
        _, logdet = slogdet(cov)
    invc = inv(cov)
    exponent = -0.5 * diff.T @ invc @ diff
    norm = -0.5 * (d * np.log(2*np.pi) + logdet)
    return np.exp(norm + exponent)

# -----------------------------
# 2) Simple K-Means (for init)
# -----------------------------
def kmeans(X, K, tol=1e-4, max_iters=500):
    N, D = X.shape
    rng = np.random.default_rng(0)
    centroids = X[rng.choice(N, K, replace=False)]
    labels = np.zeros(N, dtype=int)
    for it in range(max_iters):
        dists = np.linalg.norm(X[:, None, :] - centroids[None, :, :], axis=2)
        new_labels = np.argmin(dists, axis=1)
        new_centroids = np.array([X[new_labels == k].mean(axis=0) if np.any(new_labels==k) else centroids[k] 
                                  for k in range(K)])
        if np.linalg.norm(new_centroids - centroids) < tol:
            labels = new_labels
            centroids = new_centroids
            break
        centroids = new_centroids
        labels = new_labels
    return centroids, labels

# -----------------------------
# 3) GMM EM until convergence (returns params + logliks)
# -----------------------------
def gmm_em(X, K, tol=1e-4, safeguard_max_iters=1000):
    """
    Fit GMM to X with K components.
    Returns: means (K,D), covs (K,D,D), weights (K,), loglik_history (list)
    """
    N, D = X.shape
    means, init_labels = kmeans(X, K)
    covs = np.zeros((K, D, D))
    weights = np.zeros(K)
    for k in range(K):
        Xk = X[init_labels == k]
        if len(Xk) < 2:
            covs[k] = np.cov(X.T) + 1e-6*np.eye(D)
        else:
            covs[k] = np.cov(Xk, rowvar=False) + 1e-6*np.eye(D)
        weights[k] = max(len(Xk) / N, 1e-6)
    weights /= weights.sum()

    loglik_history = []
    prev_ll = -np.inf
    it = 0
    while True:
        # E-step: responsibilities
        resp = np.zeros((N, K))
        for k in range(K):
            # vectorize gaussian evaluations
            resp[:, k] = weights[k] * np.array([gaussian_pdf(x, means[k], covs[k]) for x in X])
        row_sums = resp.sum(axis=1, keepdims=True)
        # avoid division by zero
        row_sums[row_sums == 0] = 1e-12
        resp = resp / row_sums

        # compute log-likelihood (stable)
        ll = np.sum(np.log(row_sums + 1e-12))
        loglik_history.append(ll)

        # check convergence
        if it > 0 and abs(ll - prev_ll) < tol:
            break
        prev_ll = ll

        # M-step
        Nk = resp.sum(axis=0)  # effective counts
        for k in range(K):
            if Nk[k] < 1e-8:
                # reinitialize dead component
                means[k] = X[np.random.randint(0, N)]
                covs[k] = np.cov(X.T) + 1e-6*np.eye(D)
                weights[k] = 1.0 / N
                continue
            means[k] = (resp[:, k][:, None] * X).sum(axis=0) / Nk[k]
            diff = X - means[k]
            covs[k] = (resp[:, k][:, None, None] * np.einsum('ni,nj->nij', diff, diff)).sum(axis=0) / Nk[k]
            covs[k] += 1e-6 * np.eye(D)
            weights[k] = Nk[k] / N
        weights /= weights.sum()

        it += 1
        if it >= safeguard_max_iters:
            # safety: stop after many iterations
            break

    return means, covs, weights, loglik_history

# -----------------------------
# 4) Naive Bayes GMM (multi-class)
# -----------------------------
class NaiveBayesGMM:
    def __init__(self, n_components=2):
        self.n_components = n_components
        self.models = {}      # cls -> (means, covs, weights)
        self.priors = {}      # cls -> prior
        self.logliks = {}     # cls -> loglik_history (training)

    def fit(self, X, y):
        classes, counts = np.unique(y, return_counts=True)
        N = len(y)
        for cls in classes:
            Xc = X[y == cls]
            means, covs, weights, loglik_hist = gmm_em(Xc, self.n_components)
            self.models[cls] = (means, covs, weights)
            self.priors[cls] = len(Xc) / N
            self.logliks[cls] = loglik_hist

    def predict_proba(self, X):
        # returns array (N, n_classes) of unnormalized posteriors
        classes = list(self.models.keys())
        N = X.shape[0]
        post = np.zeros((N, len(classes)))
        for i, cls in enumerate(classes):
            means, covs, weights = self.models[cls]
            K = means.shape[0]
            # compute p(x|cls)
            px = np.zeros(N)
            for k in range(K):
                px += weights[k] * np.array([gaussian_pdf(x, means[k], covs[k]) for x in X])
            post[:, i] = px * self.priors[cls]
        # normalize to get probs
        denom = post.sum(axis=1, keepdims=True)
        denom[denom==0] = 1e-12
        probs = post / denom
        return probs, list(self.models.keys())

    def predict(self, X):
        probs, classes = self.predict_proba(X)
        idx = np.argmax(probs, axis=1)
        return np.array([classes[i] for i in idx])

    # Save/load
    def save(self, folder="gmm_saved"):
        os.makedirs(folder, exist_ok=True)
        np.save(os.path.join(folder, "n_components.npy"), np.array([self.n_components]))
        np.save(os.path.join(folder, "classes.npy"), np.array(list(self.models.keys()), dtype=object))
        np.save(os.path.join(folder, "priors.npy"), self.priors)
        for cls, (means, covs, weights) in self.models.items():
            np.save(os.path.join(folder, f"class_{cls}_means.npy"), means)
            np.save(os.path.join(folder, f"class_{cls}_covs.npy"), covs)
            np.save(os.path.join(folder, f"class_{cls}_weights.npy"), weights)
        # save logliks
        np.save(os.path.join(folder, "logliks.npy"), self.logliks)

    def load(self, folder="gmm_saved"):
        self.n_components = int(np.load(os.path.join(folder, "n_components.npy"))[0])
        classes = np.load(os.path.join(folder, "classes.npy"), allow_pickle=True)
        self.priors = np.load(os.path.join(folder, "priors.npy"), allow_pickle=True).item()
        self.models = {}
        for cls in classes:
            means = np.load(os.path.join(folder, f"class_{cls}_means.npy"))
            covs = np.load(os.path.join(folder, f"class_{cls}_covs.npy"))
            weights = np.load(os.path.join(folder, f"class_{cls}_weights.npy"))
            self.models[cls] = (means, covs, weights)
        self.logliks = np.load(os.path.join(folder, "logliks.npy"), allow_pickle=True).item()

# -----------------------------
# 5) Metrics: precision, recall, F1, accuracy per-class
# -----------------------------
def confusion_matrix(y_true, y_pred, classes=None):
    if classes is None:
        classes = np.unique(np.concatenate([y_true, y_pred]))
    class_to_idx = {c: i for i, c in enumerate(classes)}
    C = np.zeros((len(classes), len(classes)), dtype=int)
    for t, p in zip(y_true, y_pred):
        C[class_to_idx[t], class_to_idx[p]] += 1
    return C, classes

def classification_report(y_true, y_pred):
    C, classes = confusion_matrix(y_true, y_pred)
    TP = np.diag(C)
    FP = C.sum(axis=0) - TP
    FN = C.sum(axis=1) - TP
    TN = C.sum() - (TP + FP + FN)

    precision = np.zeros(len(classes))
    recall = np.zeros(len(classes))
    f1 = np.zeros(len(classes))
    support = C.sum(axis=1)

    for i in range(len(classes)):
        precision[i] = TP[i] / (TP[i] + FP[i]) if (TP[i] + FP[i]) > 0 else 0.0
        recall[i] = TP[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0.0
        f1[i] = (2 * precision[i] * recall[i] / (precision[i] + recall[i])) if (precision[i] + recall[i]) > 0 else 0.0

    accuracy = TP.sum() / C.sum() if C.sum() > 0 else 0.0

    report = {
        'classes': classes,
        'confusion_matrix': C,
        'accuracy': accuracy,
        'precision_per_class': precision,
        'recall_per_class': recall,
        'f1_per_class': f1,
        'mean_precision': precision.mean(),
        'mean_recall': recall.mean(),
        'mean_f1': f1.mean(),
        'support': support
    }
    return report

# -----------------------------
# 6) Plotting utilities
# -----------------------------
def plot_confusion_matrix(C, classes, cmap='Blues'):
    plt.figure(figsize=(6,5))
    plt.imshow(C, interpolation='nearest', cmap=cmap)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    for i in range(len(classes)):
        for j in range(len(classes)):
            plt.text(j, i, str(C[i,j]), ha='center', va='center', color='black')
    plt.tight_layout()
    plt.show()

def plot_density_contours(model: NaiveBayesGMM, X, classes_to_plot=None, levels=[0.001,0.01,0.05,0.1,0.2,0.4]):
    """
    Plot constant density contour for each class (mixture density p(x|class)), superposed with X.
    Works for 2D data only.
    """
    assert X.shape[1] == 2, "Contour plot only for 2D data"
    x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
    y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300), np.linspace(y_min, y_max, 300))
    grid = np.c_[xx.ravel(), yy.ravel()]

    if classes_to_plot is None:
        classes_to_plot = list(model.models.keys())
    plt.figure(figsize=(8,6))
    colors = plt.cm.get_cmap('tab10')
    for i, cls in enumerate(classes_to_plot):
        means, covs, weights = model.models[cls]
        # evaluate p(x|class)
        px = np.zeros(len(grid))
        for k in range(means.shape[0]):
            px += weights[k] * np.array([gaussian_pdf(g, means[k], covs[k]) for g in grid])
        px = px.reshape(xx.shape)
        CS = plt.contour(xx, yy, px, levels=levels, alpha=0.8, cmap='coolwarm')
        plt.clabel(CS, inline=1, fontsize=8)
    plt.scatter(X[:,0], X[:,1], c='k', s=10, alpha=0.6)
    plt.title("Constant density contours (p(x|class)) with training data")
    plt.show()

def plot_decision_regions(model: NaiveBayesGMM, X, y, resolution=200):
    assert X.shape[1] == 2, "Decision region plot only for 2D data"
    x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
    y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, resolution), np.linspace(y_min, y_max, resolution))
    grid = np.c_[xx.ravel(), yy.ravel()]
    Z = model.predict(grid)
    classes = list(model.models.keys())
    class_to_idx = {c:i for i,c in enumerate(classes)}
    Zi = np.array([class_to_idx[z] for z in Z]).reshape(xx.shape)
    plt.figure(figsize=(8,6))
    plt.contourf(xx, yy, Zi, alpha=0.3, cmap='tab10')
    plt.scatter(X[:,0], X[:,1], c=[class_to_idx[v] for v in y], cmap='tab10', edgecolor='k')
    plt.title("Decision regions with training data")
    plt.show()

def plot_loglik_iterations(model: NaiveBayesGMM):
    plt.figure(figsize=(8,6))
    for cls, hist in model.logliks.items():
        plt.plot(hist, label=f"class {cls}")
    plt.xlabel("Iterations")
    plt.ylabel("Log-Likelihood")
    plt.title("Iterations vs Log-Likelihood (per-class GMM training)")
    plt.legend()
    plt.show()

# -----------------------------
# 7) Image segmentation using GMM clusters (Dataset-2(c) style)
# -----------------------------
def gmm_segment_image(model_params, img_array):
    """
    model_params: tuple (means, covs, weights) for clustering (K clusters)
    img_array: HxW or HxWx3 numpy array; we flatten and run GMM responsibilities
    returns segmented image where each pixel replaced by cluster index (0..K-1)
    """
    means, covs, weights = model_params
    H, W = img_array.shape[:2]
    if img_array.ndim == 3:
        data = img_array.reshape(-1, 3).astype(float)
    else:
        data = img_array.reshape(-1, 1).astype(float)
    N = data.shape[0]
    K = means.shape[0]
    resp = np.zeros((N, K))
    for k in range(K):
        resp[:, k] = weights[k] * np.array([gaussian_pdf(x, means[k], covs[k]) for x in data])
    labels = np.argmax(resp, axis=1)
    seg = labels.reshape(H, W)
    return seg

# -----------------------------
# 8) Utility: run experiments for list of K values and return metrics + models
# -----------------------------
def run_experiments(X_train, y_train, X_test, y_test, K_list, n_components_per_class=None):
    """
    For each K in K_list (common K for all classes), trains a NaiveBayesGMM,
    computes test metrics, and stores log-likelihood histories.
    Returns: dict keyed by K containing model, report, and other info.
    """
    results = {}
    for K in K_list:
        print("Training for K =", K)
        model = NaiveBayesGMM(n_components=K)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        report = classification_report(y_test, y_pred)
        C, classes = confusion_matrix(y_test, y_pred)
        results[K] = {
            'model': model,
            'report': report,
            'confusion_matrix': C,
            'classes': classes,
            'y_pred': y_pred
        }
        print(f"Done K={K}: acc={report['accuracy']:.4f}, mean_f1={report['mean_f1']:.4f}")
    return results


In [ ]:
X_train, y_train, X_test, y_test, class_map = load_train_test_datasets(r'../../Dataset/Group04/NLS_Group04/')
run_experiments(X_train, y_train, X_test, y_test,[2,4,8])

In [7]:
# After training or from results dict:
best_model = results[best_K]['model']   # choose best_K based on validation or metrics
report = results[best_K]['report']
print("Accuracy:", report['accuracy'])
print("Classes:", report['classes'])
for i, cls in enumerate(report['classes']):
    print(f"Class {cls}: precision={report['precision_per_class'][i]:.3f}, recall={report['recall_per_class'][i]:.3f}, f1={report['f1_per_class'][i]:.3f}")
print("Mean precision:", report['mean_precision'])
print("Mean recall:", report['mean_recall'])
print("Mean F1:", report['mean_f1'])


NameError: name 'results' is not defined

In [11]:
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix
)
import seaborn as sns

# ==============================
# 1. Basic Gaussian PDF
# ==============================
def gaussian_pdf(x, mean, cov):
    d = len(mean)
    x_m = x - mean
    return np.exp(-0.5 * x_m @ np.linalg.inv(cov) @ x_m.T) / (
        np.sqrt((2 * np.pi) ** d * np.linalg.det(cov)) + 1e-12
    )

# ==============================
# 2. Simple K-Means (for GMM init)
# ==============================
def kmeans(X, K, max_iters=100, tol=1e-4):
    N, D = X.shape
    centroids = X[np.random.choice(N, K, replace=False)]
    for _ in range(max_iters):
        distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centroids = np.array([X[labels == k].mean(axis=0) for k in range(K)])
        if np.allclose(centroids, new_centroids, atol=tol):
            break
        centroids = new_centroids
    return centroids, labels

# ==============================
# 3. GMM using EM
# ==============================
def gmm_em(X, K, tol=1e-4):
    N, D = X.shape
    means, labels = kmeans(X, K)
    covs = np.array([np.cov(X[labels == k].T) + 1e-6*np.eye(D) for k in range(K)])
    weights = np.array([np.mean(labels == k) for k in range(K)])

    log_likelihood_list = []
    prev_log_likelihood = -np.inf

    while True:
        # ---------- E-Step ----------
        resp = np.zeros((N, K))
        for k in range(K):
            for n in range(N):
                resp[n, k] = weights[k] * gaussian_pdf(X[n], means[k], covs[k])
        resp /= resp.sum(axis=1, keepdims=True)

        # ---------- M-Step ----------
        Nk = resp.sum(axis=0)
        for k in range(K):
            means[k] = (resp[:, k][:, None] * X).sum(axis=0) / Nk[k]
            diff = X - means[k]
            covs[k] = (resp[:, k][:, None, None] *
                       np.einsum('ni,nj->nij', diff, diff)).sum(axis=0) / Nk[k]
            covs[k] += 1e-6 * np.eye(D)
            weights[k] = Nk[k] / N

        # ---------- Log Likelihood ----------
        log_likelihood = np.sum(np.log(
            np.sum([weights[k] *
                    np.array([gaussian_pdf(x, means[k], covs[k]) for x in X])
                    for k in range(K)], axis=0)
        ))
        log_likelihood_list.append(log_likelihood)

        if abs(log_likelihood - prev_log_likelihood) < tol:
            break
        prev_log_likelihood = log_likelihood

    return means, covs, weights, log_likelihood_list

# ==============================
# 4. Train per-class GMM
# ==============================
def train_gmm_classifier(X_train, y_train, K=3):
    classes = np.unique(y_train)
    models = {}
    for c in classes:
        Xc = X_train[y_train == c]
        means, covs, weights, ll_list = gmm_em(Xc, K)
        models[c] = (means, covs, weights)
    return models

# ==============================
# 5. Predict class labels
# ==============================
def predict_gmm_classifier(X, models):
    N = X.shape[0]
    classes = list(models.keys())
    scores = np.zeros((N, len(classes)))
    for i, c in enumerate(classes):
        means, covs, weights = models[c]
        for k in range(len(weights)):
            scores[:, i] += weights[k] * np.array(
                [gaussian_pdf(x, means[k], covs[k]) for x in X]
            )
    return np.argmax(scores, axis=1)

# ==============================
# 6. Evaluation & Visualization
# ==============================
def evaluate_and_plot(dataset_name, X_train, y_train, X_test, y_test,
                      y_pred, models, log_likelihood_list, results_dir="results_gmm"):
    os.makedirs(results_dir, exist_ok=True)

    # ---- Metrics ----
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average=None)
    rec = recall_score(y_test, y_pred, average=None)
    f1 = f1_score(y_test, y_pred, average=None)

    print(f"\n=== {dataset_name} ===")
    print(f"Accuracy: {acc:.4f}")
    for i in range(len(prec)):
        print(f"Class {i}: Precision={prec[i]:.3f}, Recall={rec[i]:.3f}, F1={f1[i]:.3f}")
    print(f"Mean Precision={prec.mean():.3f}, Mean Recall={rec.mean():.3f}, Mean F1={f1.mean():.3f}")

    # ---- Confusion Matrix ----
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix - {dataset_name}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.savefig(os.path.join(results_dir, f"confusion_{dataset_name}.png"))
    plt.close()

    # ---- Log Likelihood ----
    plt.figure()
    plt.plot(log_likelihood_list)
    plt.title(f"Log Likelihood Convergence - {dataset_name}")
    plt.xlabel("Iterations")
    plt.ylabel("Log Likelihood")
    plt.grid(True)
    plt.savefig(os.path.join(results_dir, f"loglikelihood_{dataset_name}.png"))
    plt.close()

    # ---- Decision Region (2D only) ----
    if X_train.shape[1] == 2:
        x_min, x_max = X_train[:, 0].min()-1, X_train[:, 0].max()+1
        y_min, y_max = X_train[:, 1].min()-1, X_train[:, 1].max()+1
        xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                             np.linspace(y_min, y_max, 200))
        grid = np.c_[xx.ravel(), yy.ravel()]
        pred_grid = predict_gmm_classifier(grid, models)
        plt.contourf(xx, yy, pred_grid.reshape(xx.shape), alpha=0.4, cmap='rainbow')
        plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, s=30, edgecolor='k')
        plt.title(f"Decision Regions - {dataset_name}")
        plt.savefig(os.path.join(results_dir, f"decision_{dataset_name}.png"))
        plt.close()

# ==============================
# 7. Example Run (synthetic data)
# ==============================
if __name__ == "__main__":
    
    X_train, y_train, X_test, y_test, class_map = load_train_test_datasets(r'../../Dataset/Group04/NLS_Group04/')
    dataset_name = "Dataset1"
    models = train_gmm_classifier(X_train, y_train, K=3)
    y_pred = predict_gmm_classifier(X_test, models)

    # Collect log-likelihoods from one class for plotting
    _, _, _, ll_list = gmm_em(X_train[y_train == 0], K=3)

    evaluate_and_plot(dataset_name, X_train, y_train, X_test, y_test,
                      y_pred, models, ll_list)



=== Dataset1 ===
Accuracy: 1.0000
Class 0: Precision=1.000, Recall=1.000, F1=1.000
Class 1: Precision=1.000, Recall=1.000, F1=1.000
Class 2: Precision=1.000, Recall=1.000, F1=1.000
Mean Precision=1.000, Mean Recall=1.000, Mean F1=1.000


In [12]:
def plot_density_contours(X_train, y_train, models, dataset_name, results_dir="results_gmm"):
    import matplotlib.pyplot as plt
    os.makedirs(results_dir, exist_ok=True)

    # Only 2D data can be visualized
    if X_train.shape[1] != 2:
        print("Density contour plots only supported for 2D data.")
        return

    x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
    y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    grid = np.c_[xx.ravel(), yy.ravel()]

    plt.figure(figsize=(8,6))

    # Plot density for each class
    for c, (means, covs, weights) in models.items():
        z_total = np.zeros(grid.shape[0])
        for k in range(len(weights)):
            z_total += weights[k] * np.array([gaussian_pdf(p, means[k], covs[k]) for p in grid])
        plt.contour(xx, yy, z_total.reshape(xx.shape), levels=5, alpha=0.7, cmap='cool', linestyles='--')

    # Plot training points
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, s=30, edgecolor='k')
    plt.title(f"Constant Density Contours - {dataset_name}")
    plt.xlabel("X1")
    plt.ylabel("X2")
    plt.savefig(os.path.join(results_dir, f"contours_{dataset_name}.png"))
    plt.close()
    print(f"Saved constant density contours for {dataset_name}")


In [13]:
# For Dataset-1 (2D)
plot_density_contours(X_train, y_train, models, "Dataset1")

# # For Dataset-2(a) (2D)
# plot_density_contours(X_train2a, y_train2a, models2a, "Dataset2a")


Saved constant density contours for Dataset1


NameError: name 'X_train2a' is not defined